In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 이미지와 CSV 파일 불러오기

In [ ]:
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import KFold
import tensorflow as tf


path="/content/drive/MyDrive/petfinder-pawpularity-score"
train=pd.read_csv(path+'/train.csv')
test=pd.read_csv(path+'/test.csv')

In [ ]:
train["file_path"] = train["Id"].apply(lambda x:path +"/train/"+ x + ".jpg")
test["file_path"] = test["Id"].apply(lambda x:path +"/test/"+ x + ".jpg")

In [ ]:
train["file_path"][:5]

0    /content/drive/MyDrive/petfinder-pawpularity-s...
1    /content/drive/MyDrive/petfinder-pawpularity-s...
2    /content/drive/MyDrive/petfinder-pawpularity-s...
3    /content/drive/MyDrive/petfinder-pawpularity-s...
4    /content/drive/MyDrive/petfinder-pawpularity-s...
Name: file_path, dtype: object

# 이미지 전처리
- 이미지 크기 128 x 128로 변환
- JPEG로 인코딩 된 이미지를 uint8 텐서로 디코딩
- 255로 나누어 픽셀값 정규화 



In [ ]:
image_size = 128
def preprocess(image_url):
    image_string = tf.io.read_file(image_url)
    image = tf.image.decode_jpeg(image_string, channels=3)
    image = tf.cast(image, tf.float32) / 255.0
    image = tf.image.resize(image, (image_size, image_size))
    return image
    

image.png


## 이미지 전처리(train, test) 

In [ ]:
x_train=[]
for i in train['file_path']:
    x1=preprocess(i)
    x_train.append(x1)

In [ ]:
test2=[]
for i in test['file_path']:
    x1=preprocess(i)
    test2.append(x1)
test2=np.array(test2)

#모델 학습


In [ ]:
x_train=np.array(x_train)
y_train=train['Pawpularity']
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x_train,y_train,test_size=0.2)

In [ ]:
inputs=keras.Input(shape=(128,128,3))
x=inputs
x=keras.layers.Conv2D(filters=4,kernel_size=3,strides=2,padding='same',activation='relu')(x)
x=keras.layers.Conv2D(filters=8,kernel_size=3,strides=2,padding='same',activation='relu')(x)
x=keras.layers.Conv2D(filters=16,kernel_size=3,strides=2,padding='same',activation='relu')(x)
x=keras.layers.Conv2D(filters=32,kernel_size=3,strides=2,padding='same',activation='relu')(x)
x=keras.layers.Flatten()(x)
x=keras.layers.Dense(1024, activation = "relu")(x)
x=keras.layers.Dropout(0.5)(x)
output = tf.keras.layers.Dense(1)(x)
model = tf.keras.Model(inputs=inputs, outputs=output)


In [ ]:
early_stop = tf.keras.callbacks.EarlyStopping(
    patience=5
    )
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    factor=0.1,
    patience=2, 
    min_lr=1e-9
    )
callbacks = [early_stop,reduce_lr]

In [ ]:
model.compile(loss='mse', optimizer='Adam', metrics=[tf.keras.metrics.RootMeanSquaredError(name="rmse"), "mae", "mape"])

In [ ]:
history = model.fit(x_train,y_train, epochs=25,batch_size=32,validation_data = (x_test,y_test),callbacks=callbacks)

Epoch 1/25
248/248 [==============================] - 12s 46ms/step - loss: 498.6767 - rmse: 22.3311 - mae: 16.6474 - mape: 80.4053 - val_loss: 444.2657 - val_rmse: 21.0776 - val_mae: 14.8641 - val_mape: 63.2936
Epoch 2/25
248/248 [==============================] - 11s 45ms/step - loss: 437.4355 - rmse: 20.9150 - mae: 15.6050 - mape: 80.7010 - val_loss: 432.5914 - val_rmse: 20.7988 - val_mae: 16.2206 - val_mape: 80.4542
Epoch 3/25
248/248 [==============================] - 11s 46ms/step - loss: 436.5288 - rmse: 20.8933 - mae: 15.6365 - mape: 79.9707 - val_loss: 453.6171 - val_rmse: 21.2983 - val_mae: 14.8228 - val_mape: 61.2979
Epoch 4/25
248/248 [==============================] - 11s 45ms/step - loss: 436.5308 - rmse: 20.8933 - mae: 15.6568 - mape: 80.4055 - val_loss: 424.9342 - val_rmse: 20.6139 - val_mae: 15.4336 - val_mape: 73.2537
Epoch 5/25
248/248 [==============================] - 11s 45ms/step - loss: 432.9048 - rmse: 20.8064 - mae: 15.5865 - mape: 80.2285 - val_loss: 424.9246

In [ ]:
cnn_pred=model.predict(test2)
cnn_pred

array([[32.845005],
       [34.812817],
       [32.30809 ],
       [33.947346],
       [33.07913 ],
       [33.14959 ],
       [33.424202],
       [32.028748]], dtype=float32)